In [22]:

class Empty(Exception):
    """Error attempting to access an element from an empty container"""
    pass

class LinkedQueue:
    """FIFO queue implementation using a singly linked list for storage."""
    
    # ------------------------- nested Node class --------------------------
    class _Node:
        """Lightweight, nonpublic class for storing a singly linked node."""
        __slots__ = '_element', '_next'

        def __init__(self, element, next): 
            self._element = element 
            self._next = next
    
    def __init__(self):
        """Create an empty queue."""
        self._head = None 
        self._tail = None 
        self._size = 0

    def __len__(self):
        """Return the number of elements in the queue."""
        return self._size 
    
    def is_empty(self):
        """Return True if the queue is empty."""
        return self._size == 0
    
    def first(self):
        """Return (but do not remove) the element at the front of the queue."""
        if self.is_empty():
            raise Empty('Queue is empty')
        return self._head._element

    def dequeue(self):
        """Remove and return the first element of the queue (i.e., FIFO)
        Raise Empty exception if the queue is empty.
        """
        if self.is_empty():
            raise Empty('Queue is empty')
        answer = self._head._element 
        self._head = self._head._next
        self._size -= 1
        if self.is_empty():                      # special case as queue is empty
            self._tail = None                    # removed head had been the tail
        return answer       

    def enqueue(self, e):
        """Add an element to the back of queue"""   
        newest = self._Node(e, None)
        if self.is_empty():
            self._head = newest 
        else:
            self._tail._next = newest
        self._tail = newest
        self._size += 1
    
    def traverse(self, result, node = None):
        if not node:
            node = self._head
        result.append(node._element)
        if node._next:
            self.traverse(result, node._next)
        return result

class Tree: 
    """Abstract base class representing a tree structure"""

    # ---------- nested Position class -------------- 
    class Position:
        """An abstraction representing the location of a single element."""

        def element(self):
            """Return the element stored at this Position."""
            raise NotImplementedError('must be implemented by subclass.')

        def __eq__(self, other):
            """Return True if other Position represents the same location."""
            raise NotImplementedError('must be implemented by subclass.')
        
        def __ne__(self, other):
            """Return True if other does not represent the same location."""
            return not(self == other)
    
    # ------------- abstract methods that concrete subclass must support -----------
    def root(self):
        '''Return Position representing the tree's root (or None if empty)'''
        raise NotImplementedError('must be implmented by subclass')

    def parent(self, p):
        """Return Position representing p's parent(or None if p is root)."""
        raise NotImplementedError('must be implmented by subclass')
    
    def num_children(self, p):
        """Return the number of children that Position p has."""
        raise NotImplementedError('must be implmented by subclass')
    
    def children(self, p):
        """Generate an iteration of Positions representing p's children."""
        raise NotImplementedError('must be implemented by subclass')
    
    def __len__(self):
        """Return the total number of elements in the tree."""
        raise NotImplementedError('must be implemented by subclass')

    # --------- concrete methods implemented in this class ----------------
    def is_root(self, p):
        """Return True if Position p represents the root of the tree."""
        return self.root() == p 
    
    def is_leaf(self, p):
        """Return True if Position p does not have any children"""
        return self.num_children(p) == 0
    
    def is_empty(self):
        """Return True if the tree is empty."""
        return len(self) == 0
    
    def __iter__(self):
        """Generate an iteration of the tree's elements."""
        for p in self.positions():
            yield p.element()
    
    # --------------- Preorder Traversal --------------
    def positions(self):
        """Generate an iteration of the tree's positions."""
        return self.preorder()          # return entire preorder iteration

    def _subtree_preorder(self, p): 
        """Generate a preorder iteration of positions in subtree rooted at p."""
        yield p                        # visit p before its subtrees
        for c in self.children(p):
            for other in self._subtree_preorder(c):
                yield other
    
    def preorder(self): # generator
        """Generate a preorder iteration of positions in subtree rooted at p."""
        if not self.is_empty():
            for p in self._subtree_preorder(self.root()):        # start recursion
                yield p

    # --------------- Postorder Traversal --------------
    def postorder(self):
        """Generate a postorder iteration of positions in the tree."""
        if not self.is_empty():
            for p in self._subtree_postorder(self.root()):
                yield p
    
    def _subtree_postorder(self, p):
        """Generate a postorder iteration of positions in subtree rooted at p."""
        for c in self.children(p):                        # for each child c
            for other in self._subtree_postorder(c):     # do postorder of c's subtree
                yield other                              # yielding each to our caller
        yield p
    
    # -------------- Breadth-First Traversal -------------
    def breadthfirst(self):
        """Generate a breadth-first iteration of the positions of the tree."""
        if not self.is_empty():
            fringe = LinkedQueue()            # known positions not yet yielded
            fringe.enqueue(self.root())       # starting with the root
            while not fringe.is_empty():
                p = fringe.dequeue()          # remove from font of the queue 
                yield p                       # report this position 
                for c in self.children(p):
                    fringe.enqueue(c)

class BinaryTree(Tree):
    """Abstract base class representing a binary tree structure."""
    # ------------------- additional abstract methods ------------------
    def left(self, p):
        """Return a Position representing p's left child.
        Return None if p does not have a left child.
        """
        raise NotImplementedError('must be implemented by subclass')
    
    def right(self, p):
        """Return a Position representing p's right child.
        Return None if p does not have a right child.
        """
        raise NotImplementedError('must be implemented by subclass')
    
    # ------------------ concrete methods implemented in this class ------------------
    def sibling(self, p):
        """Return a Position representing p's sibling (or None if no sibling)."""
        parent = self.parent(p)
        if parent is None:
            return None 
        else:
            if p == self.left(parent):
                return self.right(parent)
            else:
                return self.left(parent)
    
    def children(self, p):
        """Generate an iteration of Positions representing p's children."""
        if self.left(p) is not None:
            yield self.left(p)
        if self.right(p) is not None:
            yield self.right(p)

    # ------- Inorder Traversal ------------ 
    def inorder(self):
        """Generate an inorder iteration of positions in the tree."""
        if not self.is_empty():
            for p in self._subtree_inorder(self.root()):
                yield p 

    def _subtree_inorder(self, p):
        """Generate an inorder iteration of positions in subtree rooted at p."""
        if self.left(p) is not None:        # if left child exists, traverse its subtree 
            for other in self._subtree_inorder(self.left(p)):
                yield other 
        yield p                             # visit p between its subtrees 
        if self.right(p) is not None:       # if right child exists, traverse its subtree 
            for other in self._subtree_inorder(self.right(p)):
                yield other 

    # override inherited version of the tree's positions.
    def positions(self):
        """Generate an iteration of the tree's positions."""
        return self.inorder()


class LinkedBinaryTree(BinaryTree):
    """Linked representation of a binary tree structure."""

    class _Node:           # Lightweight, nonpublic class for storing a node 
        __slots__ = '_element', '_parent', '_left', '_right'
        
        def __init__(self, element, parent=None, left=None, right=None):
            self._element = element 
            self._parent = parent 
            self._left = left 
            self._right = right 
    
    class Position(BinaryTree.Position):
        """An abstraction representing the location of a single element."""
        
        def __init__(self, container, node):
            """Constructure should not be invoked by user."""
            self._container = container 
            self._node = node 
        
        def element(self):
            return self._node._element
        
        def __eq__(self, other):
            """Return True if other is a Position representing the same location."""
            return type(other) is type(self) and other._node is self._node
    
    def _validate(self, p):
        """Return associated node, if position is valid."""
        if not isinstance(p, self.Position):
            raise TypeError('p must be proper Position type')
        
        if p._container is not self:
            raise ValueError('p does not belong to this container')
        
        if p._node._parent is p._node: # convention for deprecated nodes
            raise ValueError('p is no longer valid')
        return p._node

    def _make_position(self, node):
        """Return Position instance for given node (or None if no node)."""
        return self.Position(self, node) if node is not None else None
    
    # --------------- binary tree constructure -----------------------
    def __init__(self):
        """Create an initially empty binary tree"""
        self._root = None 
        self._size = 0
    
    # --------------- public accessors --------------------------
    def __len__(self):
        return self._size 
    
    def root(self):
        """Return the root Position of the tree (or None if tree is empty)."""
        return self._make_position(self._root)
    
    def parent(self, p):
        """Return the Position of p's parent (or None if p is root)"""
        node = self._validate(p)
        return self._make_position(node._parent)
    
    def left(self, p):
        """Return the Position of p's left child (or None if no left child)."""
        node = self._validate(p)
        return self._make_position(node._left)
    
    def right(self, p):
        """Return the Position of p's right child (or None if no right child)."""
        node = self._validate(p)
        return self._make_position(node._right)

    def num_children(self, p):
        """Return the number of children of Position p."""
        node = self._validate(p)
        count = 0
        if node._left is not None:
            count += 1 
        if node._right is not None:
            count += 1 
        return count 

    # --------- nonpublic update methods -------------------------
    def _add_root(self, e):
        """Place element e at the root of an empty tree and return new Position.
        Raise ValueError if tree nonempty.
        """
        if self._root is not None: raise ValueError('Root exists')
        self._size = 1 
        self._root = self._Node(e)
        return self._make_position(self._root)
    
    def _add_left(self, p, e):
        node = self._validate(p)
        if node._left is not None: raise ValueError('Left child exists')
        self._size += 1
        node._left = self._Node(e, node)
        return self._make_position(node._left)
    
    def _add_right(self, p, e):
        node = self._validate(p)
        if node._right is not None: raise ValueError('Right child exists')
        self._size += 1
        node._right = self._Node(e, node)
        return self._make_position(node._right)
    
    def _replace(self, p, e):
        """Replace the element at position p with e, and return old element."""
        node = self._validate(p)
        old = node._element 
        node._element = e 
        return old
    
    def _delete(self, p):
        """Delete the node at Position p, and replace it with its child, if any. 
        Return the element that had been stored at Position p.
        Raise ValueError if POsition p is in valid or p has two hildren.
        """
        node = self._validate(p)
        if self.num_children(p) == 2: raise ValueError('p has two children.')
        child = node._left if node._left else node._right 
        if child is not None:
            child._parent = node._parent 
        
        if node is self._root:
            self._root = child
        else:
            parent = node._parent 
            if node is parent._left:
                parent._left = child 
            else: 
                parent._right = child
        
        self._size -= 1
        node._parent = node           # convention for deprecated node
        return node._element
    
    def _attach(self, p, t1, t2):
        """Attach trees t1 and t2 as left and right subtrees of external p."""
        node = self._validate(p)
        if not self.is_leaf(p): raise ValueError('position must be leaf')
        if not type(self) is type(t1) is type(t2): # all 3 trees must be same type
            raise TypeError('Tree types must match.')
        self._size += len(t1) + len(t2)
        if not t1.is_empty():           # attached t1 as left subtree of node 
            t1._root._parent = node 
            node._left = t1._root
            t1._root = None             # set t1 instance to empty
            t1._size = 0 
        if not t2.is_empty():           # attached t2 as right subtree of node 
            t2._root._parent = node 
            node._right = t2._root 
            t2._root = None             # set t2 instance to empty
            t2._size = 0

    def depth(self, p):
        if self.is_root(p):
            return 0 
        else:
            return 1 + self.depth(self.parent(p))
        
    def _height2(self, p): # O(n)
        if self.is_leaf(p):
            return 0 
        else:
            return 1 + max(self._height2(c) for c in self.children(p))
    # R-8.4 What is the running time of a call to T._height2(p) when called on a position p
    # distinct from the root of T -> O(n - c) (c means the number of nodes in depth)

    def left_child_sum(self):
        """Generate a postorder iteration of positions in the tree."""
        if not self.is_empty():
            for p in self._subtree_postorder(self.root()):
                yield 1
    
    def _subtree_postorder(self, p):
        """Generate a postorder iteration of positions in subtree rooted at p."""
        for c in self.children(p):                        # for each child c
            for other in self._subtree_postorder(c):     # do postorder of c's subtree
                if c == self.left(p):
                    yield 1                             # yielding each to our caller
        yield 1
            

            
        
        
# ------------- Some applications ----------------
def preorder_indent(T, p, d):
    """Print preorder representation of subtree of T rooted at p at depth d."""
    print(2 * d * ' ' + str(p.element()))              # use depth for indentation
    for c in T.children(p):
        preorder_indent(T, c, d+1)                     # child depth is d+1

lb_tree = LinkedBinaryTree()
lb_tree._add_root(5)
lb_tree._add_left(lb_tree.root(), 3)
lb_tree._add_right(lb_tree.root(), 4)
lb_tree._add_left(lb_tree.left(lb_tree.root()), 6)
lb_tree._add_right(lb_tree.left(lb_tree.root()), 2)
lb_tree._add_left(lb_tree.right(lb_tree.root()), 7)
lb_tree._add_right(lb_tree.right(lb_tree.root()), 8)
# inorder
# print([e._node._element for e in lb_tree.inorder()])
# preorder
# print([e._node._element for e in lb_tree.preorder()])
# postorder
# print([e._node._element for e in lb_tree.postorder()])
# print indent 
# print(preorder_indent(lb_tree, lb_tree.root(), 0))
# left child count 
print('left child num is ->', sum(n for n in lb_tree.left_child_sum()))

# Draw an arithmetic expression tree that has four external nodes, 
# storing the numbers 1, 5, 6, and 7 (with each number stored in a distinct external node, 
# but not necessarily in this order), and has three internal nodes, 
# each storing an operator from the set {+,−,×,/}, 
# so that the value of the root is 21. The operators may return and act on fractions, 
# and an operator may be used more than once.
ar_tree = LinkedBinaryTree()
ar_tree._add_root('-')
ar_tree._add_left(ar_tree.root(), '*')
ar_tree._add_right(ar_tree.root(), '*')
ar_tree._add_left(ar_tree.left(ar_tree.root()), 5)
ar_tree._add_right(ar_tree.left(ar_tree.root()), 6)
ar_tree._add_left(ar_tree.right(ar_tree.root()), 7)
ar_tree._add_right(ar_tree.right(ar_tree.root()), 1)
"""
Algorithm breadthfirst(T)
Initialize queue Q to contain T.root()

while Q not empty do 
    p = Q.dequeue()        {p is the oldest entry in the queue}
    perform the "visit" action for position p
    for each child c in T.children(p) do 
        Q.enqueue(c)
"""

# 8.4.3 Inorder Traversal of a Binary Tree
# (1) preorder 
# (2) postorder 
# (3) breadth-first

# Inorder traversal
# left -> root -> right
"""
Algorithm inorder(p):
    if p has a left child lc then 
        inorder(lc)                         {recursively traverse the left subtree of p}
    perform the "visit" action for position p 
    if p has a right child ls then 
        inorder(rc)                         {recursively traverse the right subtree of p}
"""

# Binary Search Trees
# S be a se whose unique elements have an order relation 
# S could be a set of integers 
# A binary search tree for S is a binary tree T

# An example of a binary search tree is shown in Figure 8.19.
# The above properties assure that an inorder traversal of a binary search tree T visits the elements in nondecreasing order 

# binary search 
def binary_search(T, n, v):
    if v == n._node._element:
        return True 
    if v < n._node._element:
        binary_search(T, T.left(n), v)
    if v > n._node._element:
        binary_search(T, T.right(n), v)
    return False


# T.positions(): Generate an iteration of all positions of tree T.
# iter(T): Generate an iteration of all elements stored within tree T.

def preorder_label(T, p, d, path):
    """Print labeled representation of subtree of T rooted at p at depth d."""
    label = '.'.join(str(j+1) for j in path)
    print(2*d*' ' + label, p.element())
    path.append(0)
    for c in T.children(p):
        preorder_label(T, c, d+1, path)
        print('path is ->', path)
        path[-1] += 1
    path.pop()

lb_tree = LinkedBinaryTree()
lb_tree._add_root("Main")
lb_tree._add_left(lb_tree.root(), "Book")
lb_tree._add_right(lb_tree.root(), "Music")
lb_tree._add_left(lb_tree.left(lb_tree.root()), "good")
lb_tree._add_right(lb_tree.left(lb_tree.root()), "finance")
lb_tree._add_left(lb_tree.right(lb_tree.root()), "piano")
lb_tree._add_right(lb_tree.right(lb_tree.root()), "violen")

# preorder_label(lb_tree, lb_tree.root(), 0, [])

def parenthesize(T, p):
    """Print parenthesized representation of subtree of T rooted at p."""
    print(p.element(), end='')                 # use of end avoids trailing newline
    if not T.is_leaf(p):
        first_time = True 
        for c in T.children(p):
            sep = ' (' if first_time else ', '
            print(sep, end='')
            first_time = False 
            parenthesize(T, c)                 # recur on child
        print(')', end='')                     # include closing parenthesis

# parenthesize(lb_tree, lb_tree.root())


# R-8.5 Describe an algorithm, relying only on the BinaryTree operations, that counts
# the number of leaves in a binary tree that are left child of their respective parent.


# What are the minimum and maximum number of internal and external nodes in an improper binary tree 
# with n nodes ?

# max number : ceil(n/2) - 1
# min number : ceil(n/4) + 1

"""
Let T be a binary tree with n positions that is realized with an array representation A, 
and let f() be the level numbering function of the positions of T, 
as given in Section 8.3.2. 
Give pseudo-code descriptions of each of the methods 
root, parent, left, right, is leaf, and is root.
"""
A = ['a', 'b', 'c', 'd', 'e', 'f', 'g']

class ArrayBianryTree:

    def __init__(self):
        self._data = []
    
    def get_index(self, n):
        return self._data.index(n)
    
    def root(self):
        if self._data:
            return self._data[0]
    
    def parent(self, n):
        lp_index = (n - 1) / 2
        rp_index = (n - 2) /2
        if isinstance(lp_index, int) and lp_index < len(self._data):
            return self._data[lp_index]
        elif isinstance(rp_index, int) and rp_index < len(self._data):
            return self._data[rp_index]

    def left(self, n):
        np = self.get_index(n)
        nl = 2*np + 1
        if nl < len(self._data):
            return self._data[nl]
    
    def right(self, n):
        np = self.get_index(n)
        nr = 2*np + 2
        if nr < len(self._data):
            return self._data[nr]
    
    def is_leaf(self, n):
        lc = 2*(self._data.index(n)) + 1
        rc = 2*(self._data.index(n)) + 2
        try:
            if self._data[lc] or self._data[rc]:
                return True
        except IndexError:
            return False

left child num is -> 3


In [24]:
[1, 2, 3].index(4)

ValueError: 4 is not in list